# Imports

In [8]:
import tarfile
import numpy as np
import codecs
import zlib
from bs4 import BeautifulSoup
import pandas as pd

import spinn3rApi_pb2 as proto_api
import protoStream_pb2 as proto_stream
from google.protobuf.internal.decoder import _DecodeVarint32

# Helper Functions

In [2]:
# Each .protostream file contains exactly 200 distinct records

def read_protostream_file(file):
    decoder = _DecodeVarint32
    
    ## get the three types of protobuf messages we expect to see
    header    = proto_stream.ProtoStreamHeader()
    delimiter = proto_stream.ProtoStreamDelimiter()
    
    ## get the header
    # length, pos = decoder(file, pos) => pos is starting point, length is the length of the record
    length, pos = decoder(file, 0)
    header.ParseFromString(file[pos:pos + length])
    # print(header)
    ## should check its contents

    pos += length
    data_rows = []

    while pos < len(file):
        length, pos = decoder(file, pos)
        delimiter.ParseFromString(file[pos:pos + length])
        if delimiter.delimiter_type == delimiter.END:
            break
        elif delimiter.delimiter_type == delimiter.ENTRY:
            pos += length
            length, pos = decoder(file, pos)
            entry = proto_api.Entry()
            entry.ParseFromString(file[pos:pos + length])
            data = {}
            
            # Language
            try:
                data['Lang_Code'] = entry.feed_entry.lang[0].code
            except:
                pass
            
            # Language Probability
            try:
                data['Lang_Prob'] = entry.feed_entry.lang[0].probability
            except:
                pass
            
            # Title
            try:
                data['Post_Title'] = entry.feed_entry.title
            except:
                pass
            
            # Content
            try:
                data['Post_Content'] = BeautifulSoup(zlib.decompress(
                    entry.feed_entry.content.data), "html.parser").text
            except:
                pass
            
            # Link
            try:
                data['Post_Link'] = entry.feed_entry.link[0].href
            except:
                pass
            
            # Author Name
            try:
                data['Author_Name'] = entry.feed_entry.author[0].name
            except:
                pass
            
            # Author Link
            try:
                data['Author_Link'] = entry.feed_entry.author[0].link[0].href
            except:
                pass
            
            # Date & Time
            try:
                data['Datetime'] = entry.feed_entry.last_published
            except:
                pass
            
            # Identifier
            try:
                data['Identifier'] = entry.feed_entry.identifier
            except:
                pass
            
            # Spam Probability
            try:
                data['Spam'] = entry.feed_entry.spam_probability
            except:
                pass
            
            # Publisher Type
            try:
                data['Type'] = entry.source.publisher_type
            except:
                pass
            
            # Category
            try:
                data['Category'] = entry.feed_entry.category
            except:
                pass
            
            data_rows.append(data)
        pos += length
    return data_rows

# Main

In [ ]:
PREFIX = '/Volumes/Khalil Mrini/'
FILES = '01-14-OTHER 01-14-SOCIAL_MEDIA 02-11-OTHER 02-11-SOCIAL_MEDIA'.split(' ')
EXTENSION = '.tar.gz'

data_rows = []

for file_name in FILES:
    print('Opening', file_name, '...')
    tar = tarfile.open(PREFIX + file_name + EXTENSION, "r:gz")
    members = tar.getmembers()
    member_count = len(members)
    print('There are {} files in {}.'.format(member_count, file_name + EXTENSION))
    for member_index in range(member_count):
        print(member_index, 'out of', member_count, end='\r', flush=True)
        file_data = tar.extractfile(members[member_index])
        if file_data is not None:
            content = file_data.read()
            data_rows.extend(read_protostream_file(content))

In [6]:
len(data_rows)

3582070

In [11]:
pd.DataFrame(data_rows[:-1]).to_json('s.json')

OSError: [Errno 22] Invalid argument

# Reading a Single `.protostream` File

In [ ]:
# Each .protostream file contains exactly 200 distinct records

def read_protostream_entry(file):
    decoder = _DecodeVarint32
    
    ## get the three types of protobuf messages we expect to see
    header    = proto_stream.ProtoStreamHeader()
    delimiter = proto_stream.ProtoStreamDelimiter()
    
    ## get the header
    # length, pos = decoder(file, pos) => pos is starting point, length is the length of the record
    length, pos = decoder(file, 0)
    header.ParseFromString(file[pos:pos + length])
    # print(header)
    ## should check its contents

    pos += length
    data_rows = []

    while pos < len(file):
        length, pos = decoder(file, pos)
        delimiter.ParseFromString(file[pos:pos + length])
        if delimiter.delimiter_type == delimiter.END:
            break
        elif delimiter.delimiter_type == delimiter.ENTRY:
            pos += length
            length, pos = decoder(file, pos)
            entry = proto_api.Entry()
            entry.ParseFromString(file[pos:pos + length])
            data_rows.append(entry)
        pos += length
    return data_rows

# Selecting filtering Keywords

Let's analyse the [Wikipedia page](https://en.wikipedia.org/wiki/Tunisian_Revolution) for the thematic we are interested in:

In [14]:
import wikipedia
from collections import Counter
from nltk.tokenize import *
from nltk.corpus import stopwords
from nltk.collocations import *
import string
import numpy as np
import operator

In [25]:
def get_KeyWords(wiki_page, num_keywords, language, stop_words_language):
    #download wikipedia page
    wikipedia.set_lang(language)
    wiki = wikipedia.page(wiki_page)

    #initialize token's counter
    tokenizer = RegexpTokenizer(r'\w+')
    wiki_tokens = [token for token in tokenizer.tokenize(wiki.content.lower()) 
                   if token not in set(stopwords.words(stop_words_language))]
    count = Counter(wiki_tokens)
    
    #remove stop words
    #sorted_count = sorted(count.items(), key=operator.itemgetter(1), reverse=True)
    
    #return [i[0] for i in sorted_count][:num_keywords]
    return count.most_common(num_keywords)

In [26]:
get_KeyWords('Western Sahara', 10, 'en', 'english')

[('sahara', 81),
 ('morocco', 78),
 ('western', 75),
 ('moroccan', 57),
 ('sahrawi', 53),
 ('territory', 46),
 ('polisario', 44),
 ('spanish', 39),
 ('area', 25),
 ('un', 24)]